In [174]:
print('Takumi House & Kailen Santos')

Takumi House & Kailen Santos


In [175]:
from matplotlib import pyplot as plt
import matplotlib as mp
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import sklearn.manifold
from scipy import cluster
import seaborn as sns
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
import requests
import json
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
import re
import json
import pickle
%matplotlib inline

In [176]:
##load in pickled dataframe of all the reviews

#try:
#    with open('df_reviews','rb') as f:
#        df_reviews = pickle.load(f)
        
#except FileNotFoundError:
#    with open('df_reviews','wb') as f:
#        pickle.dump(df_reviews, f)

In [177]:
##load in pickled stemmed data for the review text
try:
    with open('stemmed_data','rb') as f:
        stemmed_data = pickle.load(f)
        
except FileNotFoundError:
    stemmed_data = [" ".join(SnowballStemmer("english", ignore_stopwords=True).stem(word)  
        for sent in sent_tokenize(message)
        for word in word_tokenize(sent))
        for message in df_reviews['text']]
    
    with open('stemmed_data','wb') as f:
        pickle.dump(stemmed_data, f)

In [178]:
df = pd.read_csv('Restaurant_Inspections.csv',header=0)
# df = df[df.City == 'Las Vegas']
# df = (df[df['Inspection Result'] == 'Closed with Fees']).append(df[df['Inspection Result'] == 'Closed without Fees'])

df.shape

#with open ('Resaurant_Inspections.csv') as file:
#   for line in file:
#      if (line[6]=='Las Vegas' && line[]

/Users/takumihouse/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(123919, 23)

In [179]:
df = df[df.City == 'Las Vegas']
df.shape

(94700, 23)

In [180]:
categories = df['Category Name'].unique().tolist()
print(categories)

['Pantry', 'Special Kitchen', 'Garde Manger', 'Kitchen Bakery', 'Bar / Tavern', 'Buffet', 'Snack Bar', 'Restaurant', 'Banquet Support', 'Portable Unit', 'Banquet Kitchen', 'Portable Bar', 'Meat/Poultry/Seafood', 'Vegetable Prep', 'Caterer', 'Barbeque', 'Confection', 'Concessions', 'Produce Market', 'Institutional Food Service', 'Elementary School Kitchen', 'Self-Service Food Truck', 'Bakery Sales', 'Food Trucks / Mobile Vendor', 'Grocery Store Sampling', 'Beer Bar', 'Childcare Kitchens', "Farmer's Market", 'Main Kitchen', 'Frozen Meat Sales']


In [181]:
categories.remove('Elementary School Kitchen')
categories.remove('Grocery Store Sampling')
categories.remove('Farmer\'s Market')
categories.remove('Caterer')
categories.remove('Concessions')
categories.remove('Institutional Food Service')
categories.remove('Banquet Support')
categories.remove('Snack Bar')

In [182]:
df = df[df['Category Name'].isin(categories)]
df.shape

(82656, 23)

In [183]:
coord_strings = df['Location 1'].tolist()

In [184]:
import ast
xcoords = []
ycoords = []
for coord in coord_strings:
    xcoords.append(ast.literal_eval(coord)[0])
    ycoords.append(ast.literal_eval(coord)[1])

In [185]:
count = 0
for idx, ycoord in enumerate(ycoords):
    if ycoord > 0:
        ycoords[idx] = ycoord * -1
        count += 1
print('{} coordinates fixed.'.format(count))

510 coordinates fixed.


In [186]:
df['Coordinates'] = list(zip(xcoords, ycoords))
df.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Employee ID,Inspection Type,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates
11,DA0001715,PR0000426,CAESARS AUGUSTUS BUTLER PANTRY,CAESARS PALACE HOTEL & CASINO,Pantry,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000417,Routine Inspection,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
12,DA0001716,PR0000426,CAESARS AUGUSTUS BUTLER PANTRY,CAESARS PALACE HOTEL & CASINO,Pantry,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000567,Routine Inspection,2,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
13,DA0001728,PR0000427,CAESARS DIAMOND LOUNGE KITCHEN,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,8,...,EE7000417,Routine Inspection,4,A,NaN,Compliant,14,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
14,DA0001773,PR0000434,CAESARS BAKERY WAREWASH,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000417,Routine Inspection,7,A,NaN,Compliant,14232831,01/22/2014 02:03:44 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
15,DA0001776,PR0000434,CAESARS BAKERY WAREWASH,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000392,Routine Inspection,2,A,NaN,Compliant,NaN,01/22/2014 02:03:44 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"


In [187]:
coordinates_list = df['Coordinates'].tolist()
xcoords = []
ycoords = []
for coordinates in coordinates_list:
    xcoords.append(coordinates[0])
    ycoords.append(coordinates[1])
df['X'] = xcoords
df['Y'] = ycoords

In [188]:
df = df[df['Y'] < -114.5]
df = df[df['Y'] > -115.6]
df_clean = df.reset_index(drop = True)

In [189]:
coordinates_list = df_clean['Coordinates'].tolist()
xcoords = []
ycoords = []
for coordinates in coordinates_list:
    xcoords.append(coordinates[0])
    ycoords.append(coordinates[1])

In [190]:
df = df_clean[df_clean['Date Current']==df_clean['Inspection Date']]
# df['XY'] = list(zip(df.X, df.Y))
df.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates,X,Y
60,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,4,...,4,A,NaN,Compliant,114,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)",36.116156,-115.175058
187,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,10,...,10,A,NaN,Compliant,64114,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
243,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,3,...,3,A,NaN,Compliant,13,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
306,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,Las Vegas,Nevada,89101-5601,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1713005, -115.1462312)","(36.1713005, -115.1462312)",36.171301,-115.146231
410,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,Las Vegas,Nevada,89101,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1800444, -115.131106)","(36.1800444, -115.131106)",36.180044,-115.131106


In [191]:
# df_sorted = df.sort(columns = 'XY', axis = 0)
# df_sorted.head(n=40)

In [192]:
df_clean = df.reset_index(drop=True)
df_clean.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates,X,Y
0,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,4,...,4,A,NaN,Compliant,114,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)",36.116156,-115.175058
1,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,10,...,10,A,NaN,Compliant,64114,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
2,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,3,...,3,A,NaN,Compliant,13,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
3,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,Las Vegas,Nevada,89101-5601,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1713005, -115.1462312)","(36.1713005, -115.1462312)",36.171301,-115.146231
4,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,Las Vegas,Nevada,89101,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1800444, -115.131106)","(36.1800444, -115.131106)",36.180044,-115.131106


In [193]:
# df_agg = df_clean.groupby('Restaurant Name',as_index=False).agg(lambda x: x.tolist())
df_agg = df_clean.copy()

In [194]:
df_agg.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates,X,Y
0,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,4,...,4,A,NaN,Compliant,114,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)",36.116156,-115.175058
1,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,10,...,10,A,NaN,Compliant,64114,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
2,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,3,...,3,A,NaN,Compliant,13,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
3,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,Las Vegas,Nevada,89101-5601,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1713005, -115.1462312)","(36.1713005, -115.1462312)",36.171301,-115.146231
4,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,Las Vegas,Nevada,89101,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1800444, -115.131106)","(36.1800444, -115.131106)",36.180044,-115.131106


In [195]:
dfcategories = df_agg.columns
dfcategories

Index(['Serial Number', 'Permit Number', 'Restaurant Name', 'Location Name',
       'Category Name', 'Address', 'City', 'State', 'Zip', 'Current Demerits',
       'Current Grade', 'Date Current', 'Inspection Date', 'Inspection Time',
       'Employee ID', 'Inspection Type', 'Inspection Demerits',
       'Inspection Grade', 'Permit Status', 'Inspection Result', 'Violations',
       'Record Updated', 'Location 1', 'Coordinates', 'X', 'Y'],
      dtype='object')

In [196]:
# drop unnecessary columns for categorization step later
# df_agg = df_agg.drop(['Serial Number'], axis = 1)
df_agg = df_agg.drop(['City'], axis = 1)
df_agg = df_agg.drop(['State'], axis = 1)
df_agg = df_agg.drop(['Zip'], axis = 1)
df_agg = df_agg.drop(['Inspection Time'], axis = 1)
df_agg = df_agg.drop(['Employee ID'], axis = 1)
df_agg = df_agg.drop(['Inspection Type'], axis = 1)
df_agg = df_agg.drop(['Location 1'], axis = 1)
df_agg = df_agg.drop(['Coordinates'], axis = 1)
df_agg = df_agg.drop(['Record Updated'], axis = 1)
df_agg = df_agg.drop(['Inspection Date'], axis = 1)

In [197]:
df_agg.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,4,A,05/17/2010 12:00:00 AM,4,A,NaN,Compliant,114,36.116156,-115.175058
1,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,10,A,07/30/2010 12:00:00 AM,10,A,NaN,Compliant,64114,36.120601,-115.176838
2,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,3,A,07/30/2010 12:00:00 AM,3,A,NaN,Compliant,13,36.120601,-115.176838
3,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,0,A,03/16/2010 12:00:00 AM,0,A,NaN,Compliant,NaN,36.171301,-115.146231
4,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,0,A,05/04/2011 12:00:00 AM,0,A,NaN,Compliant,NaN,36.180044,-115.131106


In [198]:
df_agg.shape

(12926, 16)

In [199]:
# dfcategories = df_agg.columns
# for x in dfcategories[1:]:
#     for y in range(len(df_agg[x])):
#         df_agg[x][y] = list(set(df_agg[x][y]))
#         df_agg[x][y] = df_agg[x][y][0]

In [200]:
df_agg.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,4,A,05/17/2010 12:00:00 AM,4,A,NaN,Compliant,114,36.116156,-115.175058
1,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,10,A,07/30/2010 12:00:00 AM,10,A,NaN,Compliant,64114,36.120601,-115.176838
2,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,3,A,07/30/2010 12:00:00 AM,3,A,NaN,Compliant,13,36.120601,-115.176838
3,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,0,A,03/16/2010 12:00:00 AM,0,A,NaN,Compliant,NaN,36.171301,-115.146231
4,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,0,A,05/04/2011 12:00:00 AM,0,A,NaN,Compliant,NaN,36.180044,-115.131106


In [201]:
try:
    with open('df_agg_revs','rb') as f:
        df_agg_revs = pickle.load(f)
        
except FileNotFoundError:
    with open('df_agg_revs','wb') as f:
        pickle.dump(df_agg_revs, f)

In [202]:
sick_ids = []
for x in df_agg_revs['business_id'].items():
    sick_ids.append(x[1])
print(sick_ids)

['enaf8BVkZ8IOvjZjn0YoDg', 'k2CdgSzyVR7b-L75tfXHUA', 'XDZP-6QpzRAghJaYRLtGrw', 'RD83slMtxNqRZ1cIe6b4Ng', 'yxxLFua918P53BQADGCSCw', 'UbSuXdtTmdzV8QzU6t011g', '4uPreeZvTDkso81i-e849Q', 'Urw6wXWzGAuTOUfcaqXoPQ', '40xXIs_WIOvrhtbS7QJ1ZQ', 'rGvkljWlP7ysvLEUucoyqA', '27ADmOieSUZfbiU15als7w', 'C7o7x_Gz-iNGNGNPZx28bw', '5nCo6iU2uysEthHY5m72Jw', 'Rw7_FiN3ccqw-sADshjBZQ', '6gJj6LSZY5HsOhaYpIvQxw', 'vTjwSPEpaPdqUtjPk7lRyA', 'JwztCPClRL1QdyjncBUbuQ', '_LjNO1X-jFr_HWSfnkTP_Q', 'FKBxlola5rfzHQ7DfMg2ZA', '5kOYuJtRjAUWZGDkP8vnRw', 's-SBFtf4ONTuLofxgp2_gw', 'PC7CxO4fZJV_OcI16oDYkg', '7q1FpSXbE6XtLNg518pxDA', '9Bnc9YP0n5cR57xkP6fQmQ', 'QpwIRDQTw-p78XZ2GiS6Zw', 'aQeyyfz_C6wg3L0Ep5-N3A', 'BEDbyHfEjh-_qu2-r1uz5Q', 'ME8gcI4BXjmFq7zEMtk3ag', '8Nsh8u0zHYegI6XT1L39Lg', 'jcJLwFit9lwYCiDBV7fuqA', 'pzF-KpQF8nRMyTUNu5jp2g', 'cXyrea2I01hwPI1rNkODfA', 'xNcqwYAUeVOhe3KNC4Xjaw', 'tHfJ2aHCIT52dwarPON7LA', '0jLy2J-6omAKhBDzrNHj9g', 'ySPLtCZKvU0g5vM7w7bS4Q', 'Kt_1Xa10qpzkxt0ln-ICvA', 'lkQ8gupLmpKZic9NOO8LCQ', 'oGIrd1goDH

In [203]:
file = 'yelp_academic_dataset_business.json'
business_locations = {}
addresses = []
names=[]
with open(file) as f:
    for line in f:
        r_line = json.loads(line)
        if r_line['business_id'] in set(sick_ids):
            business_locations.update({r_line['business_id']:(r_line['full_address']).rsplit('\n', 5)[0].lower()})
            addresses.append(r_line['full_address'].rsplit('\n', 5)[0].lower())
            names.append(r_line['name'].lower())
addresses = np.asarray(addresses)
df_agg_revs['address']=addresses
df_agg_revs['name']=names
#print(business_locations)

In [204]:
df_agg_revs.head()

,business_id,stars,text,votes,address,name
0,enaf8BVkZ8IOvjZjn0YoDg,"[5, 5, 5, 3, 5, 5, 5, 3, 5, 3, 2, 4, 4, 5, 4, ...","[This place used to be Mashisoyo. New owners, ...","[{'cool': 5, 'useful': 9, 'funny': 2}, {'cool'...",1916 village center cir,chianti cafe
1,k2CdgSzyVR7b-L75tfXHUA,"[5, 3, 1, 2, 2]","[At one point in late 2009, I was going to thi...","[{'cool': 0, 'useful': 2, 'funny': 1}, {'cool'...",2737 las vegas blvd n,rigos taco
2,XDZP-6QpzRAghJaYRLtGrw,"[2, 2, 4, 4, 3, 4, 3, 3, 2, 2, 4]","[First, let me say that Italian eateries appea...","[{'cool': 0, 'useful': 0, 'funny': 0}, {'cool'...",2312 e craig rd,mulligans bar & grill
3,RD83slMtxNqRZ1cIe6b4Ng,"[4, 4, 3]",[The Pullman Grille is one of the hidden trea...,"[{'cool': 0, 'useful': 1, 'funny': 0}, {'cool'...",3100 e lake mead blvd,domino's pizza
4,yxxLFua918P53BQADGCSCw,"[5, 5, 3, 5, 2, 3, 4, 2, 3, 4, 2, 3, 2, 4, 5, ...",[Pizza here made my night... Good people and g...,"[{'cool': 0, 'useful': 2, 'funny': 0}, {'cool'...",1824 e lake mead boulevard,carl's jr restaurants


CODE FOR READING IN PICKLED OVERLAPPED FILES

In [205]:
try:
    with open('df_overlapped_inspections','rb') as f:
        df_overlapped_inspections = pickle.load(f)
        
except FileNotFoundError:
    print('can\'t find it')

In [206]:
try:
    with open('df_overlapped_reviews','rb') as f:
        df_overlapped_reviews = pickle.load(f)
        
except FileNotFoundError:
    print('can\'t find it')

In [207]:
df_lv_reviews = df_agg_revs.copy()
df_lv_reviews = df_lv_reviews[['business_id','address', 'name']]
df_lv_reviews.head()

,business_id,address,name
0,enaf8BVkZ8IOvjZjn0YoDg,1916 village center cir,chianti cafe
1,k2CdgSzyVR7b-L75tfXHUA,2737 las vegas blvd n,rigos taco
2,XDZP-6QpzRAghJaYRLtGrw,2312 e craig rd,mulligans bar & grill
3,RD83slMtxNqRZ1cIe6b4Ng,3100 e lake mead blvd,domino's pizza
4,yxxLFua918P53BQADGCSCw,1824 e lake mead boulevard,carl's jr restaurants


In [208]:
df_lv_reviews.shape

(4910, 3)

In [209]:
df_reviews_search = df_lv_reviews.set_index('address')
df_reviews_search.head()

,business_id,name
address,,
1916 village center cir,enaf8BVkZ8IOvjZjn0YoDg,chianti cafe
2737 las vegas blvd n,k2CdgSzyVR7b-L75tfXHUA,rigos taco
2312 e craig rd,XDZP-6QpzRAghJaYRLtGrw,mulligans bar & grill
3100 e lake mead blvd,RD83slMtxNqRZ1cIe6b4Ng,domino's pizza
1824 e lake mead boulevard,yxxLFua918P53BQADGCSCw,carl's jr restaurants


In [210]:
df_reviews_search.shape

(4910, 2)

In [211]:
df_lv_violations = df_agg.copy()
df_lv_violations = df_lv_violations[['Serial Number','Restaurant Name', 'Address']]
df_lv_violations['Restaurant Name'] = df_lv_violations['Restaurant Name'].str.lower()
df_lv_violations['Address'] = df_lv_violations['Address'].str.lower()
df_lv_violations.head()

,Serial Number,Restaurant Name,Address
0,DA0003413,caesars seasonal venus pool bar deleted,3570 s las vegas blvd
1,DA0009343,mirage 1 oak small bar 2 original deleted,3400 s las vegas blvd
2,DA0011310,mirage 1 oak multilevel bar 4 deleted,3400 s las vegas blvd
3,DA0017310,golden gate beverage cart,1 fremont st
4,DA0023482,aramark nacho cart,850 n las vegas blvd


In [212]:
df_lv_violations.shape

(12926, 3)

In [213]:
df_violations_search = df_lv_violations.set_index('Address')
df_violations_search.head()
# len(df_lv_violations['Address'].unique())

,Serial Number,Restaurant Name
Address,,
3570 s las vegas blvd,DA0003413,caesars seasonal venus pool bar deleted
3400 s las vegas blvd,DA0009343,mirage 1 oak small bar 2 original deleted
3400 s las vegas blvd,DA0011310,mirage 1 oak multilevel bar 4 deleted
1 fremont st,DA0017310,golden gate beverage cart
850 n las vegas blvd,DA0023482,aramark nacho cart


In [214]:
df_violations_search.shape

(12926, 2)

In [215]:
# bad_add = []
# for add_list in df_lv_violations['Address'].tolist():
#     if (len(set(add_list)) > 1):
#         bad_add.append(add_list)      

In [216]:
# def mergeDFs():
df_merged = pd.merge(df_reviews_search, df_violations_search, right_index=True, left_index=True)
# for index, row in df_reviews_search.iterrows():
#     if index in df_violations_search.index:
#         match_rows = df_violations_search.loc[:index]
df_merged.columns = ['business_id', 'yelp_name', 'Serial Number', 'hi_name']
df_merged.head()
        

,business_id,yelp_name,Serial Number,hi_name
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0510924,plaza hotel stuffed
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0621500,plaza steakhouse
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0621507,plaza steakhouse - bar
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0866334,plaza hotel veg prep / pantry
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0889534,plaza hotel derby lounge


In [217]:
df_merged.shape

(13773, 4)

In [218]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [219]:
df_merged['sim_score']= df_merged.apply(lambda row: similar(a = row['yelp_name'], b = row['hi_name']), axis = 1)

In [220]:
df_reviews_search.loc['1 s main st']

,business_id,name
address,,
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway
1 s main st,tPmjepYrjiKyfGOpedyDBw,center stage restaurant
1 s main st,TN49j3bUJr3fuKCJkCatSA,mcdonald's
1 s main st,aGulqXnFmpOh_5Qh96uBng,pop up pizza
1 s main st,RbeQtptWcltLiS_4MKijOw,zabas mexican grill
1 s main st,3trWuWYn8gv3bieAXrBToQ,beer garden


In [221]:
df_reviews_search.loc['1 s main st'].shape

(6, 2)

In [222]:
df_violations_search.loc['1 s main st']

,Serial Number,Restaurant Name
Address,,
1 s main st,DA0510924,plaza hotel stuffed
1 s main st,DA0621500,plaza steakhouse
1 s main st,DA0621507,plaza steakhouse - bar
1 s main st,DA0866334,plaza hotel veg prep / pantry
1 s main st,DA0889534,plaza hotel derby lounge
1 s main st,DA0943583,plaza hotel in room service
1 s main st,DA0945204,hash house a go go - main kitchen/storage
1 s main st,DA0945213,plaza hotel main kitchen
1 s main st,DA1013638,island sushi & hawaiian grill bar


In [223]:
df_violations_search.loc['1 s main st'].shape

(45, 2)

In [224]:
df_merged.loc['1 s main st']

,business_id,yelp_name,Serial Number,hi_name,sim_score
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0510924,plaza hotel stuffed,0.160000
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0621500,plaza steakhouse,0.181818
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0621507,plaza steakhouse - bar,0.285714
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0866334,plaza hotel veg prep / pantry,0.114286
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0889534,plaza hotel derby lounge,0.066667
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0943583,plaza hotel in room service,0.060606
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0945204,hash house a go go - main kitchen/storage,0.127660
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA0945213,plaza hotel main kitchen,0.066667
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA1013638,island sushi & hawaiian grill bar,0.205128
1 s main st,XOyrNbUoagGU2s8EXc1I9w,subway,DA1038257,gigi's,0.166667


In [225]:
df_merged_sorted = df_merged.sort(columns = 'sim_score', axis = 0, ascending=False)
df_merged_sorted.head(n=200)

/Users/takumihouse/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,business_id,yelp_name,Serial Number,hi_name,sim_score
2040 n nellis blvd,tCXIc3z5epTRqlZPqhPqsQ,arandas taqueria,DA1438424,arandas taqueria,1.0
3981 e sunset rd,o_jm_d4Gpqubfo9AMUixCg,capriotti's sandwich shop,DA1556780,capriotti's sandwich shop,1.0
2893 w lake mead blvd,zTVjTJYlRW8-cT5IQZpkBg,mega tom's burgers,DA1670825,mega tom's burgers,1.0
2885 e charleston blvd,5YgXxBUzhf3qh8F8wYSqsA,pizza patron,DA1051853,pizza patron,1.0
3945 s durango dr,I_40De6H0fEIApspLkQ14g,naked fish's sushi & grill,DA1653919,naked fish's sushi & grill,1.0
3220 n jones blvd,nX38puAVP3qV7AjYANkObA,soul foo young,DA1114175,soul foo young,1.0
3950 n tenaya way,eF3ckn8vFfP73cOZRDFykw,manhattan pizza iv,DA1686793,manhattan pizza iv,1.0
3950 schiff dr,l8JIAsWIO2-0g7jftcD3nw,hunan jiu jia,DA1283644,hunan jiu jia,1.0
2850 e tropicana ave,1rjOg5oc2Mic0ssqj2ex7A,casa di amore,DA1648571,casa di amore,1.0
2840 e tropicana ave,MikwfuKo3AvdRaM97SQVzg,bentley's coffee,DA0636559,bentley's coffee,1.0


In [226]:
df_sim_threshold = df_merged_sorted[df_merged_sorted['sim_score'] > 0.69]
# df[df.a > 1]
df_sim_threshold.sort(columns = 'sim_score', axis = 0, ascending=True).head(n=20)

/Users/takumihouse/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,business_id,yelp_name,Serial Number,hi_name,sim_score
313 n nellis blvd,I3XHOUk6EIBbt1b11tblQA,hibachi grill & supreme buffet,DA1673643,hibachi grill - sushi bar,0.690909
5045 w tropicana ave,jOuERtVf7QePnK9ZcdH5XA,qdoba mexican grill,DA1664454,qdoba mexican grill tropicana / deca,0.690909
4945 w flamingo rd,-sV52FN-D-I808tyRPEvwg,mcdonald's,DA1639205,mcdonalds #17022,0.692308
7530 w washington ave,Fh6OndE60vzvyhCX1Wg7Aw,mcdonald's,DA1654263,mcdonalds #25295,0.692308
4855 s maryland pkwy,7RvQ30oHHT2W5UU_jO5Wmw,mcdonald's,DA1645261,mcdonalds #21201,0.692308
2233 n rampart blvd,kpdLm43TItSM2415L_V1FQ,pizza hut,DA1691373,pizza hut #026610,0.692308
7000 w charleston blvd,fuCKyA_ZTXVEo8BwUaeh1Q,pizza hut,DA0869192,pizza hut #026588,0.692308
5111 boulder hwy,gsQ1ZB3nzUCo93kqougigA,mcdonald's,DAQN67INY,mcdonalds #23749,0.692308
8090 blue diamond rd,707HVAsClpiHh6hgMjho-A,pizza hut,DA1700867,pizza hut #026596,0.692308
255 e flamingo rd,aLWvaNTOUr1wdOizitoOuA,tuscany gardens,DA1557599,tuscany edr,0.692308


In [228]:
df_sim_threshold.shape

(1233, 5)

In [230]:
biz_id_serial = dict(zip(df_sim_threshold['business_id'], df_sim_threshold['Serial Number']))

In [260]:
df_overlapped_reviews.shape

(1245, 7)

In [272]:
# df_test = df_overlapped_reviews.reset_index()
df_test.head()
# df_test.set_index('Serial Number')

,index,business_id_x,stars,text,votes,name,business_id_y,Serial Number
0,1 s main st,aGulqXnFmpOh_5Qh96uBng,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...",[Again attention to detail- serving diners sim...,"[{'cool': 1, 'useful': 0, 'funny': 0}, {'cool'...",pop up pizza,aGulqXnFmpOh_5Qh96uBng,DAP1Z5FBW
1,1 s main st,RbeQtptWcltLiS_4MKijOw,"[4, 1, 3, 5, 4, 5, 5, 5, 5, 5, 5, 5, 2, 3, 5, ...",[An office favorite. One of the many location...,"[{'cool': 0, 'useful': 0, 'funny': 0}, {'cool'...",zabas mexican grill,RbeQtptWcltLiS_4MKijOw,DA1649914
2,100 n city pkwy,310xHIFxZKqthaPlkQ0_dg,"[3, 1, 1, 1]",[I went on a Saturday around 6 PM and was in a...,"[{'cool': 3, 'useful': 3, 'funny': 0}, {'cool'...",jason's deli,310xHIFxZKqthaPlkQ0_dg,DAC0MOPEC
3,1000 e charleston blvd,ScSwwoFcIC0mf1vU-6p8eQ,"[4, 5, 3, 2, 5, 5, 5, 4, 2, 5, 2, 5, 4, 5, 2, ...",[It's a mystery when Ice Monster actually open...,"[{'cool': 5, 'useful': 6, 'funny': 2}, {'cool'...",esmeralda's cafe,ScSwwoFcIC0mf1vU-6p8eQ,DA1691555
4,1000 n main st,CJlD7-4QHoK29bE5nH5aWA,"[5, 2, 4, 4, 4, 4]",[I have been to this restuartant twice - both ...,"[{'cool': 1, 'useful': 0, 'funny': 0}, {'cool'...",la piazza,CJlD7-4QHoK29bE5nH5aWA,DA1600775


In [255]:
df_agg_test = df_agg.set_index('Serial Number')

In [261]:
df_agg_test.head()

,Permit Number,Restaurant Name,Location Name,Category Name,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
Serial Number,,,,,,,,,,,,,,
DA0003413,PR0000492,caesars seasonal venus pool bar deleted,CAESARS PALACE HOTEL & CASINO,Portable Unit,4,A,05/17/2010 12:00:00 AM,4,A,NaN,Compliant,114,36.116156,-115.175058
DA0009343,PR0000728,mirage 1 oak small bar 2 original deleted,MIRAGE HOTEL & CASINO,Bar / Tavern,10,A,07/30/2010 12:00:00 AM,10,A,NaN,Compliant,64114,36.120601,-115.176838
DA0011310,PR0000730,mirage 1 oak multilevel bar 4 deleted,MIRAGE HOTEL & CASINO,Bar / Tavern,3,A,07/30/2010 12:00:00 AM,3,A,NaN,Compliant,13,36.120601,-115.176838
DA0017310,PR0000631,golden gate beverage cart,Golden Gate Hotel & Casino,Portable Unit,0,A,03/16/2010 12:00:00 AM,0,A,NaN,Compliant,NaN,36.171301,-115.146231
DA0023482,PR0001171,aramark nacho cart,ARAMARK @ CASHMAN CENTER,Portable Unit,0,A,05/04/2011 12:00:00 AM,0,A,NaN,Compliant,NaN,36.180044,-115.131106


In [266]:
temp_dict = dict(zip(df_agg_test.index.values, df_agg_test['Inspection Grade']))

In [267]:
temp_dict

{'DA0607552': 'A',
 'DANVXPTH9': 'A',
 'DA1687390': 'A',
 'DA1456454': 'A',
 'DA0JFP81P': 'A',
 'DA1657172': 'A',
 'DA1663915': 'A',
 'DA1691070': 'A',
 'DA1640829': 'A',
 'DA1168765': 'A',
 'DA1563488': 'A',
 'DAO9QX01A': 'A',
 'DA1588233': 'A',
 'DA0384254': 'A',
 'DA1672696': 'A',
 'DA1648136': 'A',
 'DAL3USF5P': 'A',
 'DAMI648RZ': 'A',
 'DA1446429': 'A',
 'DA1541261': 'A',
 'DA1500952': 'A',
 'DA1451110': 'A',
 'DA1661769': 'A',
 'DA1655232': 'A',
 'DA1706215': 'A',
 'DA1166426': 'A',
 'DA1656273': 'A',
 'DA1554964': 'A',
 'DAO8PURBJ': 'A',
 'DAHUEMZ0D': 'B',
 'DAC76GOUH': 'A',
 'DA1640616': 'A',
 'DAIXXYLBT': 'A',
 'DA0929919': 'A',
 'DA1670825': 'A',
 'DACLSU3XJ': 'A',
 'DA1658340': 'A',
 'DA1024743': 'A',
 'DA1648480': 'A',
 'DA0589235': 'A',
 'DA1513294': 'A',
 'DA1672102': 'A',
 'DA1662210': 'A',
 'DA1450366': 'A',
 'DA1007711': 'A',
 'DAJDG0UGC': 'A',
 'DALAOVG00': 'A',
 'DA1649744': 'A',
 'DA1645316': 'A',
 'DA1563175': 'A',
 'DA1545169': 'A',
 'DA0972603': 'A',
 'DAQYMVEJS'

In [263]:
# for row in df_test.iterrows():
#     row['inspection_grade'] = df_agg_test.ix[row['Serial Number'], 'Inspection Grade']

TypeError: tuple indices must be integers or slices, not str

In [275]:
df_test['inspection_grade'] = df_test['Serial Number'].map(temp_dict)

In [276]:
df_test.head()

,index,business_id_x,stars,text,votes,name,business_id_y,Serial Number,inspection_grade
0,1 s main st,aGulqXnFmpOh_5Qh96uBng,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...",[Again attention to detail- serving diners sim...,"[{'cool': 1, 'useful': 0, 'funny': 0}, {'cool'...",pop up pizza,aGulqXnFmpOh_5Qh96uBng,DAP1Z5FBW,A
1,1 s main st,RbeQtptWcltLiS_4MKijOw,"[4, 1, 3, 5, 4, 5, 5, 5, 5, 5, 5, 5, 2, 3, 5, ...",[An office favorite. One of the many location...,"[{'cool': 0, 'useful': 0, 'funny': 0}, {'cool'...",zabas mexican grill,RbeQtptWcltLiS_4MKijOw,DA1649914,A
2,100 n city pkwy,310xHIFxZKqthaPlkQ0_dg,"[3, 1, 1, 1]",[I went on a Saturday around 6 PM and was in a...,"[{'cool': 3, 'useful': 3, 'funny': 0}, {'cool'...",jason's deli,310xHIFxZKqthaPlkQ0_dg,DAC0MOPEC,A
3,1000 e charleston blvd,ScSwwoFcIC0mf1vU-6p8eQ,"[4, 5, 3, 2, 5, 5, 5, 4, 2, 5, 2, 5, 4, 5, 2, ...",[It's a mystery when Ice Monster actually open...,"[{'cool': 5, 'useful': 6, 'funny': 2}, {'cool'...",esmeralda's cafe,ScSwwoFcIC0mf1vU-6p8eQ,DA1691555,A
4,1000 n main st,CJlD7-4QHoK29bE5nH5aWA,"[5, 2, 4, 4, 4, 4]",[I have been to this restuartant twice - both ...,"[{'cool': 1, 'useful': 0, 'funny': 0}, {'cool'...",la piazza,CJlD7-4QHoK29bE5nH5aWA,DA1600775,A


In [277]:
with open('df_fi_sorted','rb') as f:
        df_fi_sorted = pickle.load(f)

In [280]:
biz_id_inspect_grade = dict(zip(df_test.business_id_x, df_test.inspection_grade))

In [282]:
len(biz_id_inspect_grade)

1007

In [278]:
df_fi_sorted.head()

,index,business_id,stars,text,votes,fi_scores,fi_scores_avg,open,long,lat,price_range
0,4239,FX3skXQE-HwP_Cpss_nWdQ,3.5,[U love spicy food and chciken...u gonna love ...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...","[0.0, 0.0, 0.0, -11.1936, 0.0, 0.0, 0.0, 0.150...",-1.380450,False,-115.304009,36.193609,2
1,3031,yBJgkT7950epzcHduXo6iA,3.5,[A little girl sits happily for a picture with...,"[{'useful': 4, 'funny': 0, 'cool': 1}, {'usefu...","[0.0, -12.2232, 3.0288000000000017, 0.0, 0.0, ...",-1.313486,True,-115.107831,36.209626,1
2,4724,OnhYvrZGjvzGtq39fCcVlw,4.0,[The food was alright. Meat quality and select...,"[{'useful': 3, 'funny': 0, 'cool': 0}, {'usefu...","[0.0, 0.0, 0.0, 0.0, 0.0, -23.9904, -3.7752, 0...",-0.945948,True,-115.117796,36.240425,2
3,4796,I5vURIFijxnW2sdaDl7bpw,3.5,[I beg you please....please...please don't go ...,"[{'useful': 5, 'funny': 5, 'cool': 2}, {'usefu...","[-5.3999999999999995, 0.0, 0.15000000000000002...",-0.714286,True,-115.105675,36.196344,None
4,3517,sTYVn-UblwI6P-bwDvcvRw,2.5,[I think Mama hates me. \n\nI've eaten at Ma...,"[{'useful': 2, 'funny': 3, 'cool': 0}, {'usefu...","[-3.3936000000000006, -0.99, 0.2832, 0.0, 0.05...",-0.450044,True,-115.121042,36.196263,1


In [279]:
df_fi_sorted.shape

(4910, 11)

In [283]:
df_fi_sorted['inspection_grade'] = df_fi_sorted['business_id'].map(biz_id_inspect_grade)

In [284]:
df_fi_sorted.head()

,index,business_id,stars,text,votes,fi_scores,fi_scores_avg,open,long,lat,price_range,inspection_grade
0,4239,FX3skXQE-HwP_Cpss_nWdQ,3.5,[U love spicy food and chciken...u gonna love ...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...","[0.0, 0.0, 0.0, -11.1936, 0.0, 0.0, 0.0, 0.150...",-1.380450,False,-115.304009,36.193609,2,NaN
1,3031,yBJgkT7950epzcHduXo6iA,3.5,[A little girl sits happily for a picture with...,"[{'useful': 4, 'funny': 0, 'cool': 1}, {'usefu...","[0.0, -12.2232, 3.0288000000000017, 0.0, 0.0, ...",-1.313486,True,-115.107831,36.209626,1,NaN
2,4724,OnhYvrZGjvzGtq39fCcVlw,4.0,[The food was alright. Meat quality and select...,"[{'useful': 3, 'funny': 0, 'cool': 0}, {'usefu...","[0.0, 0.0, 0.0, 0.0, 0.0, -23.9904, -3.7752, 0...",-0.945948,True,-115.117796,36.240425,2,NaN
3,4796,I5vURIFijxnW2sdaDl7bpw,3.5,[I beg you please....please...please don't go ...,"[{'useful': 5, 'funny': 5, 'cool': 2}, {'usefu...","[-5.3999999999999995, 0.0, 0.15000000000000002...",-0.714286,True,-115.105675,36.196344,None,NaN
4,3517,sTYVn-UblwI6P-bwDvcvRw,2.5,[I think Mama hates me. \n\nI've eaten at Ma...,"[{'useful': 2, 'funny': 3, 'cool': 0}, {'usefu...","[-3.3936000000000006, -0.99, 0.2832, 0.0, 0.05...",-0.450044,True,-115.121042,36.196263,1,NaN


In [285]:
df_fi_sorted.shape

(4910, 12)

In [287]:
df_final = df_fi_sorted.copy()

In [289]:
with open('df_final','wb') as f:
        pickle.dump(df_final, f)

In [54]:
# this is important, but need the indext as address for the merge later
# df_sim_threshold.reset_index(inplace = True)
# df_sim_threshold.columns = ['address', 'yelp_name', 'hi_name', 'sim_score']
# df_sim_threshold.head()

In [231]:
df_sim_threshold.shape

(1233, 5)

In [232]:
# df_lv_violations['Address'].map(len).max()

In [233]:
df_agg.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,4,A,05/17/2010 12:00:00 AM,4,A,NaN,Compliant,114,36.116156,-115.175058
1,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,10,A,07/30/2010 12:00:00 AM,10,A,NaN,Compliant,64114,36.120601,-115.176838
2,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,3,A,07/30/2010 12:00:00 AM,3,A,NaN,Compliant,13,36.120601,-115.176838
3,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,0,A,03/16/2010 12:00:00 AM,0,A,NaN,Compliant,NaN,36.171301,-115.146231
4,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,0,A,05/04/2011 12:00:00 AM,0,A,NaN,Compliant,NaN,36.180044,-115.131106


In [234]:
# def overlap_builder(address, row):
#     if 

In [235]:
df_agg['Address'] = df_agg['Address'].str.lower()
df_agg['Restaurant Name'] = df_agg['Restaurant Name'].str.lower()
print(df_agg.shape)
df_agg.set_index('Address', inplace = True)
print(df_agg.shape)
# df_agg_revs['address_lower'] = df_agg['Address'].str.lower()
# df_agg.head()

(12926, 16)
(12926, 15)


In [236]:
df_sim_threshold.head()

,business_id,yelp_name,Serial Number,hi_name,sim_score
2040 n nellis blvd,tCXIc3z5epTRqlZPqhPqsQ,arandas taqueria,DA1438424,arandas taqueria,1.0
3981 e sunset rd,o_jm_d4Gpqubfo9AMUixCg,capriotti's sandwich shop,DA1556780,capriotti's sandwich shop,1.0
2893 w lake mead blvd,zTVjTJYlRW8-cT5IQZpkBg,mega tom's burgers,DA1670825,mega tom's burgers,1.0
2885 e charleston blvd,5YgXxBUzhf3qh8F8wYSqsA,pizza patron,DA1051853,pizza patron,1.0
3945 s durango dr,I_40De6H0fEIApspLkQ14g,naked fish's sushi & grill,DA1653919,naked fish's sushi & grill,1.0


In [237]:
df_agg.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
Address,,,,,,,,,,,,,,,
3570 s las vegas blvd,DA0003413,PR0000492,caesars seasonal venus pool bar deleted,CAESARS PALACE HOTEL & CASINO,Portable Unit,4,A,05/17/2010 12:00:00 AM,4,A,NaN,Compliant,114,36.116156,-115.175058
3400 s las vegas blvd,DA0009343,PR0000728,mirage 1 oak small bar 2 original deleted,MIRAGE HOTEL & CASINO,Bar / Tavern,10,A,07/30/2010 12:00:00 AM,10,A,NaN,Compliant,64114,36.120601,-115.176838
3400 s las vegas blvd,DA0011310,PR0000730,mirage 1 oak multilevel bar 4 deleted,MIRAGE HOTEL & CASINO,Bar / Tavern,3,A,07/30/2010 12:00:00 AM,3,A,NaN,Compliant,13,36.120601,-115.176838
1 fremont st,DA0017310,PR0000631,golden gate beverage cart,Golden Gate Hotel & Casino,Portable Unit,0,A,03/16/2010 12:00:00 AM,0,A,NaN,Compliant,NaN,36.171301,-115.146231
850 n las vegas blvd,DA0023482,PR0001171,aramark nacho cart,ARAMARK @ CASHMAN CENTER,Portable Unit,0,A,05/04/2011 12:00:00 AM,0,A,NaN,Compliant,NaN,36.180044,-115.131106


In [238]:
# df_agg_revs.reset_index(inplace = True)
# df_agg_revs.columns = ['name', 'business_id', 'stars', 'text', 'votes', 'address']
df_agg_revs.set_index('address', inplace = True)
df_agg_revs.head()

,business_id,stars,text,votes,name
address,,,,,
1916 village center cir,enaf8BVkZ8IOvjZjn0YoDg,"[5, 5, 5, 3, 5, 5, 5, 3, 5, 3, 2, 4, 4, 5, 4, ...","[This place used to be Mashisoyo. New owners, ...","[{'cool': 5, 'useful': 9, 'funny': 2}, {'cool'...",chianti cafe
2737 las vegas blvd n,k2CdgSzyVR7b-L75tfXHUA,"[5, 3, 1, 2, 2]","[At one point in late 2009, I was going to thi...","[{'cool': 0, 'useful': 2, 'funny': 1}, {'cool'...",rigos taco
2312 e craig rd,XDZP-6QpzRAghJaYRLtGrw,"[2, 2, 4, 4, 3, 4, 3, 3, 2, 2, 4]","[First, let me say that Italian eateries appea...","[{'cool': 0, 'useful': 0, 'funny': 0}, {'cool'...",mulligans bar & grill
3100 e lake mead blvd,RD83slMtxNqRZ1cIe6b4Ng,"[4, 4, 3]",[The Pullman Grille is one of the hidden trea...,"[{'cool': 0, 'useful': 1, 'funny': 0}, {'cool'...",domino's pizza
1824 e lake mead boulevard,yxxLFua918P53BQADGCSCw,"[5, 5, 3, 5, 2, 3, 4, 2, 3, 4, 2, 3, 2, 4, 5, ...",[Pizza here made my night... Good people and g...,"[{'cool': 0, 'useful': 2, 'funny': 0}, {'cool'...",carl's jr restaurants


In [239]:
# print(df_agg_revs.shape)
# # df_agg_revs.set_index('name', inplace = True)
# print(df_agg_revs.shape)

In [240]:
df_inspections_merged = pd.merge(df_agg, df_sim_threshold, right_index=True, left_index=True)
# for index, row in df_reviews_search.iterrows():
#     if index in df_violations_search.index:
#         match_rows = df_violations_search.loc[:index]
df_inspections_merged = df_inspections_merged[df_inspections_merged['Restaurant Name'] == df_inspections_merged['hi_name']]
# df_inspections_merged.head()
df_inspections_merged.head()

,Serial Number_x,Permit Number,Restaurant Name,Location Name,Category Name,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y,business_id,yelp_name,Serial Number_y,hi_name,sim_score
1 s main st,DA1649914,PR0097511,zaba's mexican grill,ZABA'S MEXICAN GRILL,Restaurant,0,A,02/11/2016 12:00:00 AM,0,A,NaN,Compliant,2930,36.171678,-115.146414,RbeQtptWcltLiS_4MKijOw,zabas mexican grill,DA1649914,zaba's mexican grill,0.974359
1 s main st,DAP1Z5FBW,PR0099965,pop up pizza,POP UP PIZZA @ PLAZA HOTEL,Restaurant,0,A,08/20/2016 12:00:00 AM,0,A,NaN,'A' Grade,NaN,36.171658,-115.146362,aGulqXnFmpOh_5Qh96uBng,pop up pizza,DAP1Z5FBW,pop up pizza,1.000000
100 n city pkwy,DAC0MOPEC,PR0011086,jason's deli #5,Jasons Deli Molasky Center,Restaurant,9,A,08/08/2016 12:00:00 AM,9,A,NaN,'A' Grade,2122909292829302955,36.174294,-115.147705,310xHIFxZKqthaPlkQ0_dg,jason's deli,DAC0MOPEC,jason's deli #5,0.888889
1000 e charleston blvd,DA1691555,PR0007241,esmeraldas,ESMERALDAS,Restaurant,5,A,06/07/2016 12:00:00 AM,5,A,NaN,'A' Grade,209,36.158279,-115.140667,ScSwwoFcIC0mf1vU-6p8eQ,esmeralda's cafe,DA1691555,esmeraldas,0.769231
1000 n main st,DA1600775,PR0009534,la piazza bar,America's Best Value Inn,Bar / Tavern,3,A,11/04/2015 12:00:00 AM,3,A,NaN,Compliant,2911292829292956,36.181417,-115.139396,CJlD7-4QHoK29bE5nH5aWA,la piazza,DA1600775,la piazza bar,0.818182


In [241]:
df_overlapped_inspections = df_inspections_merged.drop(['yelp_name', 'hi_name', 'sim_score'], axis=1)
# df_overlapped_inspections.reset_index(inplace = True)
df_overlapped_inspections.head()
# df_overlapped_inspections.columns = ['address', 'Permit Number', 'Restaurant Name',
#                                      'Location Name', 'Category Name', 'Current Demerits',
#                                      'Current Grade', 'Date Current', 'Inspection Demerits',
#                                      'Inspection Grade', 'Permit Status', 'Inspection Result',
#                                      'Violations', 'X', 'Y']

,Serial Number_x,Permit Number,Restaurant Name,Location Name,Category Name,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y,business_id,Serial Number_y
1 s main st,DA1649914,PR0097511,zaba's mexican grill,ZABA'S MEXICAN GRILL,Restaurant,0,A,02/11/2016 12:00:00 AM,0,A,NaN,Compliant,2930,36.171678,-115.146414,RbeQtptWcltLiS_4MKijOw,DA1649914
1 s main st,DAP1Z5FBW,PR0099965,pop up pizza,POP UP PIZZA @ PLAZA HOTEL,Restaurant,0,A,08/20/2016 12:00:00 AM,0,A,NaN,'A' Grade,NaN,36.171658,-115.146362,aGulqXnFmpOh_5Qh96uBng,DAP1Z5FBW
100 n city pkwy,DAC0MOPEC,PR0011086,jason's deli #5,Jasons Deli Molasky Center,Restaurant,9,A,08/08/2016 12:00:00 AM,9,A,NaN,'A' Grade,2122909292829302955,36.174294,-115.147705,310xHIFxZKqthaPlkQ0_dg,DAC0MOPEC
1000 e charleston blvd,DA1691555,PR0007241,esmeraldas,ESMERALDAS,Restaurant,5,A,06/07/2016 12:00:00 AM,5,A,NaN,'A' Grade,209,36.158279,-115.140667,ScSwwoFcIC0mf1vU-6p8eQ,DA1691555
1000 n main st,DA1600775,PR0009534,la piazza bar,America's Best Value Inn,Bar / Tavern,3,A,11/04/2015 12:00:00 AM,3,A,NaN,Compliant,2911292829292956,36.181417,-115.139396,CJlD7-4QHoK29bE5nH5aWA,DA1600775


In [242]:
df_overlapped_inspections.shape

(1251, 17)

In [243]:
df_overlapped_inspections.head()

,Serial Number_x,Permit Number,Restaurant Name,Location Name,Category Name,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y,business_id,Serial Number_y
1 s main st,DA1649914,PR0097511,zaba's mexican grill,ZABA'S MEXICAN GRILL,Restaurant,0,A,02/11/2016 12:00:00 AM,0,A,NaN,Compliant,2930,36.171678,-115.146414,RbeQtptWcltLiS_4MKijOw,DA1649914
1 s main st,DAP1Z5FBW,PR0099965,pop up pizza,POP UP PIZZA @ PLAZA HOTEL,Restaurant,0,A,08/20/2016 12:00:00 AM,0,A,NaN,'A' Grade,NaN,36.171658,-115.146362,aGulqXnFmpOh_5Qh96uBng,DAP1Z5FBW
100 n city pkwy,DAC0MOPEC,PR0011086,jason's deli #5,Jasons Deli Molasky Center,Restaurant,9,A,08/08/2016 12:00:00 AM,9,A,NaN,'A' Grade,2122909292829302955,36.174294,-115.147705,310xHIFxZKqthaPlkQ0_dg,DAC0MOPEC
1000 e charleston blvd,DA1691555,PR0007241,esmeraldas,ESMERALDAS,Restaurant,5,A,06/07/2016 12:00:00 AM,5,A,NaN,'A' Grade,209,36.158279,-115.140667,ScSwwoFcIC0mf1vU-6p8eQ,DA1691555
1000 n main st,DA1600775,PR0009534,la piazza bar,America's Best Value Inn,Bar / Tavern,3,A,11/04/2015 12:00:00 AM,3,A,NaN,Compliant,2911292829292956,36.181417,-115.139396,CJlD7-4QHoK29bE5nH5aWA,DA1600775


In [244]:
df_reviews_merged = pd.merge(df_agg_revs, df_sim_threshold, right_index=True, left_index=True)
# for index, row in df_reviews_search.iterrows():
#     if index in df_violations_search.index:
#         match_rows = df_violations_search.loc[:index]
df_reviews_merged = df_reviews_merged[df_reviews_merged['name'] == df_reviews_merged['yelp_name']]
df_reviews_merged.head()
# df_reviews_merged.shape

,business_id_x,stars,text,votes,name,business_id_y,yelp_name,Serial Number,hi_name,sim_score
1 s main st,aGulqXnFmpOh_5Qh96uBng,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...",[Again attention to detail- serving diners sim...,"[{'cool': 1, 'useful': 0, 'funny': 0}, {'cool'...",pop up pizza,aGulqXnFmpOh_5Qh96uBng,pop up pizza,DAP1Z5FBW,pop up pizza,1.000000
1 s main st,RbeQtptWcltLiS_4MKijOw,"[4, 1, 3, 5, 4, 5, 5, 5, 5, 5, 5, 5, 2, 3, 5, ...",[An office favorite. One of the many location...,"[{'cool': 0, 'useful': 0, 'funny': 0}, {'cool'...",zabas mexican grill,RbeQtptWcltLiS_4MKijOw,zabas mexican grill,DA1649914,zaba's mexican grill,0.974359
100 n city pkwy,310xHIFxZKqthaPlkQ0_dg,"[3, 1, 1, 1]",[I went on a Saturday around 6 PM and was in a...,"[{'cool': 3, 'useful': 3, 'funny': 0}, {'cool'...",jason's deli,310xHIFxZKqthaPlkQ0_dg,jason's deli,DAC0MOPEC,jason's deli #5,0.888889
1000 e charleston blvd,ScSwwoFcIC0mf1vU-6p8eQ,"[4, 5, 3, 2, 5, 5, 5, 4, 2, 5, 2, 5, 4, 5, 2, ...",[It's a mystery when Ice Monster actually open...,"[{'cool': 5, 'useful': 6, 'funny': 2}, {'cool'...",esmeralda's cafe,ScSwwoFcIC0mf1vU-6p8eQ,esmeralda's cafe,DA1691555,esmeraldas,0.769231
1000 n main st,CJlD7-4QHoK29bE5nH5aWA,"[5, 2, 4, 4, 4, 4]",[I have been to this restuartant twice - both ...,"[{'cool': 1, 'useful': 0, 'funny': 0}, {'cool'...",la piazza,CJlD7-4QHoK29bE5nH5aWA,la piazza,DA1600775,la piazza bar,0.818182


In [271]:
df_overlapped_reviews = df_reviews_merged.drop(['yelp_name', 'hi_name', 'sim_score'], axis=1)
# df_overlapped_reviews.reset_index(inplace = True)
# df_overlapped_reviews.columns = ['address', 'business_id', 'stars',
#                                      'text', 'votes', '	name']
df_reviews_merged.head()
# df_overlapped_reviews.shape

,business_id_x,stars,text,votes,name,business_id_y,yelp_name,Serial Number,hi_name,sim_score
1 s main st,aGulqXnFmpOh_5Qh96uBng,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...",[Again attention to detail- serving diners sim...,"[{'cool': 1, 'useful': 0, 'funny': 0}, {'cool'...",pop up pizza,aGulqXnFmpOh_5Qh96uBng,pop up pizza,DAP1Z5FBW,pop up pizza,1.000000
1 s main st,RbeQtptWcltLiS_4MKijOw,"[4, 1, 3, 5, 4, 5, 5, 5, 5, 5, 5, 5, 2, 3, 5, ...",[An office favorite. One of the many location...,"[{'cool': 0, 'useful': 0, 'funny': 0}, {'cool'...",zabas mexican grill,RbeQtptWcltLiS_4MKijOw,zabas mexican grill,DA1649914,zaba's mexican grill,0.974359
100 n city pkwy,310xHIFxZKqthaPlkQ0_dg,"[3, 1, 1, 1]",[I went on a Saturday around 6 PM and was in a...,"[{'cool': 3, 'useful': 3, 'funny': 0}, {'cool'...",jason's deli,310xHIFxZKqthaPlkQ0_dg,jason's deli,DAC0MOPEC,jason's deli #5,0.888889
1000 e charleston blvd,ScSwwoFcIC0mf1vU-6p8eQ,"[4, 5, 3, 2, 5, 5, 5, 4, 2, 5, 2, 5, 4, 5, 2, ...",[It's a mystery when Ice Monster actually open...,"[{'cool': 5, 'useful': 6, 'funny': 2}, {'cool'...",esmeralda's cafe,ScSwwoFcIC0mf1vU-6p8eQ,esmeralda's cafe,DA1691555,esmeraldas,0.769231
1000 n main st,CJlD7-4QHoK29bE5nH5aWA,"[5, 2, 4, 4, 4, 4]",[I have been to this restuartant twice - both ...,"[{'cool': 1, 'useful': 0, 'funny': 0}, {'cool'...",la piazza,CJlD7-4QHoK29bE5nH5aWA,la piazza,DA1600775,la piazza bar,0.818182


In [170]:
df_merged_complete = pd.merge(df_overlapped_reviews, df_overlapped_inspections, right_index=True, left_index=True)
df_merged_complete = df_merged_complete[df_merged_complete['name'] == df_merged_complete['Restaurant Name']]
df_merged_complete.head()

,business_id_x,stars,text,votes,name,business_id_y,Serial Number,Serial Number_x,Permit Number,Restaurant Name,...,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y,business_id,Serial Number_y
1 s main st,aGulqXnFmpOh_5Qh96uBng,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...",[Again attention to detail- serving diners sim...,"[{'cool': 1, 'useful': 0, 'funny': 0}, {'cool'...",pop up pizza,aGulqXnFmpOh_5Qh96uBng,DAP1Z5FBW,DAP1Z5FBW,PR0099965,pop up pizza,...,08/20/2016 12:00:00 AM,0,A,NaN,'A' Grade,NaN,36.171658,-115.146362,aGulqXnFmpOh_5Qh96uBng,DAP1Z5FBW
10140 w tropicana ave,5lfBfk8OSiqUkxfufYTahA,"[3, 3, 2, 1, 2, 1, 2, 2, 1, 1, 1, 4]",[Really it should be 2.5 stars. They have the...,"[{'cool': 0, 'useful': 3, 'funny': 0}, {'cool'...",star mexican grill,5lfBfk8OSiqUkxfufYTahA,DA0985918,DA0985918,PR0022023,star mexican grill,...,05/24/2012 12:00:00 AM,10,A,NaN,Compliant,211214220229,36.100206,-115.313223,5lfBfk8OSiqUkxfufYTahA,DA0985918
10180 w tropicana ave,lmYaUujbYrDbc_U3Jn1Ecw,"[3, 4, 4, 4, 4, 1, 2, 4, 2]",[I just exited the freeway and there was their...,"[{'cool': 0, 'useful': 2, 'funny': 0}, {'cool'...",great harvest bread company,lmYaUujbYrDbc_U3Jn1Ecw,DASRGBBQM,DASRGBBQM,PR0021987,great harvest bread company,...,06/15/2016 12:00:00 AM,6,A,NaN,'A' Grade,22229302955,36.100788,-115.314490,lmYaUujbYrDbc_U3Jn1Ecw,DASRGBBQM
10430 bermuda rd,YNQgak-ZLtYJQxlDwN-qIg,"[5, 2, 4, 4, 4, 5, 4, 5, 5, 4, 5, 2, 3, 5, 5, ...","[If you think about it, $35 is really not that...","[{'cool': 0, 'useful': 2, 'funny': 0}, {'cool'...",roberto's taco shop,YNQgak-ZLtYJQxlDwN-qIg,DAABOF2IB,DAABOF2IB,PR0007843,roberto's taco shop,...,06/14/2016 12:00:00 AM,6,A,NaN,'A' Grade,21629302955,36.000454,-115.154476,YNQgak-ZLtYJQxlDwN-qIg,DAABOF2IB
1050 s rampart blvd,qhHwbSltc8MSm64vj3wcgg,"[2, 5, 5, 5, 4, 3, 5, 4, 5, 4, 4, 3, 3, 3, 4, ...","[It's new, so perhaps Mizumi is working out th...","[{'cool': 0, 'useful': 3, 'funny': 0}, {'cool'...",baja california restaurant & cantina,qhHwbSltc8MSm64vj3wcgg,DA1018633,DA1018633,PR0023609,baja california restaurant & cantina,...,02/10/2012 12:00:00 AM,7,A,NaN,Compliant,202230233,36.160778,-115.291410,qhHwbSltc8MSm64vj3wcgg,DA1018633


In [173]:
# df_merged_complete.head()
df_merged_complete.shape

(417, 24)

In [76]:
df_overlapped_reviews.shape

(1245, 5)

In [69]:
with open('df_overlapped_reviews','wb') as f:
    pickle.dump(df_overlapped_reviews, f)
with open('df_overlapped_inspections','wb') as f:
    pickle.dump(df_overlapped_inspections, f)